# Requirements
- Artefacts: 
    - Models are uploaded in a Cloud Storage or Repo
    - required Container are pushed to a Container Registry
- Clean Kubernetes Cluster (e.g. minikube, Microk8s, Azure AKS, AWS EKS, ...)
    - minikube start --cpus 10 --memory 17000 --kubernetes-version=v1.17.11 -p demo
- Python Dependencies: [requirements.txt](./requirements.txt)
- kubectl Access:
    - az aks get-credentials --resource-group myResourceGroup --name myAKSCluster
    - aws eks update-kubeconfig --name cluster_name

# Install KFServing Standalone
See: https://github.com/kubeflow/kfserving#install-kfserving
### Install Istio

In [1]:
%%writefile ./istio/istio_ns.yaml
apiVersion: v1
kind: Namespace
metadata:
  name: istio-system
  labels:
    istio-injection: disabled

Overwriting ./istio/istio_ns.yaml


In [2]:
!kubectl apply -f ./istio/istio_ns.yaml

namespace/istio-system created


In [3]:
%%writefile ./istio/istio-minimal-operator.yaml
apiVersion: install.istio.io/v1alpha1
kind: IstioOperator
spec:
  values:
    global:
      proxy:
        autoInject: disabled
      useMCP: false
      # The third-party-jwt is not enabled on all k8s.
      # See: https://istio.io/docs/ops/best-practices/security/#configure-third-party-service-account-tokens
      jwtPolicy: first-party-jwt
  addonComponents:
    pilot:
      enabled: false
    tracing:
      enabled: false
    kiali:
      enabled: false
    prometheus:
      enabled: false
    grafana:
      enabled: false
  components:
    ingressGateways:
      - name: istio-ingressgateway
        enabled: true
      - name: cluster-local-gateway
        enabled: true
        label:
          istio: cluster-local-gateway
          app: cluster-local-gateway
        k8s:
          service:
            type: ClusterIP
            ports:
            - port: 15020
              name: status-port
            - port: 80
              name: http2
            - port: 443
              name: https

Overwriting ./istio/istio-minimal-operator.yaml


In [4]:
import time
import platform
import subprocess

os_system = platform.system()
os_machine = platform.machine()
start = time.time()

# Install Istio

if os_system == 'Windows':
    !curl -L https://github.com/istio/istio/releases/download/1.6.2/istioctl-1.6.2-win.zip -o istioctl-1.6.2-win.zip
    !tar -xf istioctl-1.6.2-win.zip
elif os_system == 'Linux':
    if os_machine == 'AMD64':
        !curl -L https://github.com/istio/istio/releases/download/1.6.2/istioctl-1.6.2-linux-amd64.tar.gz -o istioctl-1.6.2-linux.tar.gz
    if os_machine == 'armv7l':
        !curl -L https://github.com/istio/istio/releases/download/1.6.2/istioctl-1.6.2-linux-armv7.tar.gz -o istioctl-1.6.2-linux.tar.gz
    if os_machine == 'aarch64':
        print('Not supported')
    !tar -zxvf istioctl-1.6.2-linux.tar.gz


subprocess.run(["istioctl.exe", "manifest", "apply", "-f", "./istio/istio-minimal-operator.yaml"])
end = time.time()
print(end-start)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   628  100   628    0     0   2308      0 --:--:-- --:--:-- --:--:--  2308

 11 39.4M   11 4516k    0     0  5739k      0  0:00:07 --:--:--  0:00:07 5739k
100 39.4M  100 39.4M    0     0  26.1M      0  0:00:01  0:00:01 --:--:-- 48.4M


39.71574950218201


In [5]:
start = time.time()

# Install Knative-Serving
!kubectl apply --filename https://github.com/knative/serving/releases/download/v0.18.0/serving-crds.yaml
!kubectl apply --filename https://github.com/knative/serving/releases/download/v0.18.0/serving-core.yaml
!kubectl apply --filename https://github.com/knative/net-istio/releases/download/v0.18.0/release.yaml

# Install Cert Manager
!kubectl apply --validate=false -f https://github.com/jetstack/cert-manager/releases/download/v0.15.1/cert-manager.yaml
!kubectl wait --for=condition=available --timeout=600s deployment/cert-manager-webhook -n cert-manager

# Install KFServing
!kubectl apply -f https://raw.githubusercontent.com/kubeflow/kfserving/master/install/v0.5.0/kfserving_crds.yaml
!kubectl apply -f https://raw.githubusercontent.com/kubeflow/kfserving/master/install/v0.5.0/kfserving.yaml

# Install Knative-Eventing
!kubectl apply --filename https://github.com/knative/eventing/releases/download/v0.18.0/eventing.yaml

# Install Knative-Monitoring
!kubectl apply --filename https://github.com/knative/serving/releases/download/v0.18.0/monitoring.yaml

end = time.time()
print(end-start)

customresourcedefinition.apiextensions.k8s.io/certificates.networking.internal.knative.dev created
customresourcedefinition.apiextensions.k8s.io/configurations.serving.knative.dev created
customresourcedefinition.apiextensions.k8s.io/ingresses.networking.internal.knative.dev created
customresourcedefinition.apiextensions.k8s.io/metrics.autoscaling.internal.knative.dev created
customresourcedefinition.apiextensions.k8s.io/podautoscalers.autoscaling.internal.knative.dev created
customresourcedefinition.apiextensions.k8s.io/revisions.serving.knative.dev created
customresourcedefinition.apiextensions.k8s.io/routes.serving.knative.dev created
customresourcedefinition.apiextensions.k8s.io/serverlessservices.networking.internal.knative.dev created
customresourcedefinition.apiextensions.k8s.io/services.serving.knative.dev created
customresourcedefinition.apiextensions.k8s.io/images.caching.internal.knative.dev created
namespace/knative-serving created
clusterrole.rbac.authorization.k8s.io/knat

namespace/knative-eventing created
serviceaccount/eventing-controller created
clusterrolebinding.rbac.authorization.k8s.io/eventing-controller created
clusterrolebinding.rbac.authorization.k8s.io/eventing-controller-resolver created
clusterrolebinding.rbac.authorization.k8s.io/eventing-controller-source-observer created
clusterrolebinding.rbac.authorization.k8s.io/eventing-controller-sources-controller created
clusterrolebinding.rbac.authorization.k8s.io/eventing-controller-manipulator created
serviceaccount/pingsource-mt-adapter created
clusterrolebinding.rbac.authorization.k8s.io/knative-eventing-pingsource-mt-adapter created
serviceaccount/eventing-webhook created
clusterrolebinding.rbac.authorization.k8s.io/eventing-webhook created
clusterrolebinding.rbac.authorization.k8s.io/eventing-webhook-resolver created
clusterrolebinding.rbac.authorization.k8s.io/eventing-webhook-podspecable-binding created
configmap/config-br-default-channel created
configmap/config-br-defaults created
conf

## Deploy InfluxDB with Helm

In [6]:
start = time.time()

!helm repo add influxdata https://helm.influxdata.com/
!helm repo update
!helm search repo influxdata
!helm install --name-template release-influxdb stable/influxdb

end = time.time()
print(end-start)

"influxdata" already exists with the same configuration, skipping
Hang tight while we grab the latest from your chart repositories...
...Successfully got an update from the "influxdata" chart repository
...Successfully got an update from the "prometheus-community" chart repository
...Successfully got an update from the "stable" chart repository
Update Complete. âŽˆHappy Helming!âŽˆ
NAME                          	CHART VERSION	APP VERSION	DESCRIPTION                                       
influxdata/chronograf         	1.1.24       	1.8.9.1    	Open-source web application written in Go and R...
influxdata/influxdb           	4.9.14       	1.8.4      	Scalable datastore for metrics, events, and rea...
influxdata/influxdb-enterprise	0.1.12       	1.8.0      	Run InfluxDB Enterprise on Kubernetes             
influxdata/influxdb2          	1.1.0        	2.0.4      	A Helm chart for InfluxDB v2                      
influxdata/kapacitor          	1.3.1        	1.5.4      	InfluxDB's native 

## Deploy ServiceAccount to store AWS Credentials for S3 Bucket Access

In [7]:
import os
from IPython.core.magic import register_line_cell_magic
from dotenv import load_dotenv
load_dotenv()

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))
        
AWS_ACCESS_KEY_ID = os.environ['AWS_ACCESS_KEY_ID']
AWS_SECRET_ACCESS_KEY = os.environ['AWS_SECRET_ACCESS_KEY']

In [8]:
%%writetemplate ./credentials/aws-secret_serviceaccount.yaml
apiVersion: v1
kind: Secret
metadata:
  name: aws-secret
  namespace: kfserving-test
type: Opaque
stringData:
  AWS_ACCESS_KEY_ID: {AWS_ACCESS_KEY_ID}
  AWS_SECRET_ACCESS_KEY: {AWS_SECRET_ACCESS_KEY}
---
apiVersion: v1
kind: ServiceAccount
metadata:
  name: sa
  namespace: kfserving-test
secrets:
  - name: aws-secret

In [9]:
!kubectl create ns kfserving-test
!kubectl apply -f ./credentials/aws-secret_serviceaccount.yaml

namespace/kfserving-test created
secret/aws-secret created
serviceaccount/sa created


## Deploy docker-registry secret to access the private Gitlab Container Registry

In [10]:
!kubectl create secret docker-registry gitlab \
    --docker-server=https://registry.gitlab.com/\
    --docker-username=%DOCKER_USERNAME%\
    --docker-password=%DOCKER_PASSWORD%\
    -n kfserving-test

secret/gitlab created


## Deploy Knative Broker

In [11]:
%%writefile ./broker.yaml
apiVersion: eventing.knative.dev/v1
kind: broker
metadata:
 name: product-recommender
 namespace: kfserving-test

Overwriting ./broker.yaml


In [12]:
!kubectl create -f ./broker.yaml

broker.eventing.knative.dev/product-recommender created


# Deploy Product Recommender

In [13]:
%%writefile ./tf-deployment-recommender.yaml
apiVersion: "serving.kubeflow.org/v1beta1"
kind: "InferenceService"
metadata:
  namespace: "kfserving-test"
  name: "product-recommender"
spec:
  transformer:
        containers:
        - image: registry.gitlab.com/felix.exel/container_registry/kfserving/model-performance-monitoring
          name: user-container
          imagePullPolicy: Always
        imagePullSecrets:
          - name: gitlab
  predictor:
    serviceAccountName: "sa" # service account for aws credentials
    minReplicas: 1 # if 0: replica will scale down to 0 when there are no requests
    tensorflow:
      runtimeVersion: "2.4.0" #TensorFlow Serving Version
      storageUri: "s3://bucket-fex/0/719f2437c2a147d89ab6268cf7379cda/artifacts/saved_model/tfmodel/" # subfolder must contain numbers only for tf serving
    logger:
      mode: all
      url: http://broker-ingress.knative-eventing.svc.cluster.local/kfserving-test/product-recommender

Overwriting ./tf-deployment-recommender.yaml


In [14]:
!kubectl apply -f ./tf-deployment-recommender.yaml

inferenceservice.serving.kubeflow.org/product-recommender created


# Deploy Anomaly Detection (Autoencoder)

In [15]:
%%writefile ./outlier_detection/outlier-detection.yaml
apiVersion: serving.kubeflow.org/v1beta1
kind: InferenceService
metadata:
  namespace: kfserving-test
  name: autoencoder-recommender
spec:
  transformer:
        containers:
        - image: registry.gitlab.com/felix.exel/container_registry/kfserving/outlier-detection
          name: user-container
          imagePullPolicy: Always
        imagePullSecrets:
          - name: gitlab

  predictor:
    serviceAccountName: "sa" # service account for aws credentials
    minReplicas: 1 # if 0: replica will scale down to 0 when there are no requests
    tensorflow:
      runtimeVersion: "2.4.0" #TensorFlow Serving Version
      storageUri: "s3://bucket-fex/autoencoder_recommender/d052e637a7314c14a092585baf512672/" # subfolder must contain numbers only for tf serving
      resources:
        limits:
          cpu: "3" # cloud: 3, local 8
        requests:
          cpu: "1" # cloud: 1, local 2

Overwriting ./outlier_detection/outlier-detection.yaml


In [16]:
!kubectl apply -f ./outlier_detection/outlier-detection.yaml

inferenceservice.serving.kubeflow.org/autoencoder-recommender created


### Trigger

In [17]:
%%writefile ./outlier_detection/trigger.yaml
apiVersion: eventing.knative.dev/v1
kind: Trigger
metadata:
  name: outlier-trigger
  namespace: kfserving-test
spec:
  broker: product-recommender
  filter:
    attributes:
      type: org.kubeflow.serving.inference.request
  subscriber:
    uri: http://autoencoder-recommender-transformer-default.kfserving-test/v1/models/autoencoder-recommender:predict

Overwriting ./outlier_detection/trigger.yaml


In [18]:
!kubectl apply -f ./outlier_detection/trigger.yaml

trigger.eventing.knative.dev/outlier-trigger created


# Deploy Concept Drift Detection

In [19]:
%%writefile ./concept_drift_detection/concept-drift.yaml
apiVersion: serving.kubeflow.org/v1beta1
kind: InferenceService
metadata:
  namespace: kfserving-test
  name: concept-drift
spec:
  predictor:
    maxReplicas: 1 # Concept Drift gathers all batches in one instance (pod) so it cannot be replicated
    minReplicas: 1 # if 0: replica will scale down to 0 when there are no requests
    containers:
    - image: registry.gitlab.com/felix.exel/container_registry/kfserving/concept-drift-detection
      name: user-container
      imagePullPolicy: Always
      ports:
        - containerPort: 8080
          protocol: TCP
    imagePullSecrets:
      - name: gitlab

Overwriting ./concept_drift_detection/concept-drift.yaml


In [20]:
!kubectl apply -f ./concept_drift_detection/concept-drift.yaml

inferenceservice.serving.kubeflow.org/concept-drift created


### Trigger

In [21]:
%%writefile ./concept_drift_detection/trigger.yaml
apiVersion: eventing.knative.dev/v1
kind: Trigger
metadata:
  name: concept-drift-trigger
  namespace: kfserving-test
spec:
  broker: product-recommender
  filter:
    attributes:
      type: org.kubeflow.serving.inference.request
  subscriber:
    uri: http://concept-drift-predictor-default.kfserving-test/v1/models/concept-drift:predict

Overwriting ./concept_drift_detection/trigger.yaml


In [22]:
!kubectl apply -f ./concept_drift_detection/trigger.yaml

trigger.eventing.knative.dev/concept-drift-trigger created


# Grafana

In [23]:
%%writefile ./istio/loadbalancer.yaml
apiVersion: v1
kind: Service
metadata:
  name: grafana-load-balancer
  namespace: knative-monitoring
spec:
  type: LoadBalancer
  selector:
    app: grafana
  ports:
    - protocol: TCP
      port: 3000
      targetPort: 3000

Overwriting ./istio/loadbalancer.yaml


In [24]:
!kubectl apply -f ./istio/loadbalancer.yaml

service/grafana-load-balancer created


In [25]:
cluster_type = 'aws' # 'azure', 'aws', 'local'

if cluster_type == 'azure': # azure aks
    INGRESS_HOST_LIST = !kubectl -n istio-system get service istio-ingressgateway -o jsonpath={.status.loadBalancer.ingress[0].ip}
    INGRESS_HOST =  INGRESS_HOST_LIST[0]
    INGRESS_PORT = 80
    INGRESS_PORT_SSL = 443
    GRAFANA_HOST_LIST = !kubectl -n knative-monitoring get service grafana-load-balancer -o jsonpath={.status.loadBalancer.ingress[0].ip}
    GRAFANA_HOST = GRAFANA_HOST_LIST[0]
    GRAFANA_PORT = 3000

elif cluster_type == 'aws': # aws eks
    INGRESS_HOST_LIST = !kubectl -n istio-system get service istio-ingressgateway -o jsonpath={.status.loadBalancer.ingress[0].hostname}
    INGRESS_HOST =  INGRESS_HOST_LIST[0]
    INGRESS_PORT = 80
    INGRESS_PORT_SSL = 443
    GRAFANA_HOST_LIST = !kubectl -n knative-monitoring get service grafana-load-balancer -o jsonpath={.status.loadBalancer.ingress[0].hostname}
    GRAFANA_HOST = GRAFANA_HOST_LIST[0]
    GRAFANA_PORT = 3000
    
elif cluster_type == 'local': # e.g. minikube or microk8s
    INGRESS_HOST_LIST = !kubectl get po -l istio=ingressgateway -n istio-system -o jsonpath={.items[0].status.hostIP}
    INGRESS_HOST =  INGRESS_HOST_LIST[0] #eg. '192.168.52.86'
    INGRESS_PORT_LIST = !kubectl get svc -l istio=ingressgateway -n istio-system -o jsonpath={.items[0].spec.ports[1].nodePort}
    INGRESS_PORT = int(INGRESS_PORT_LIST[0])
    INGRESS_PORT_SSL_LIST = !kubectl get svc -l istio=ingressgateway -n istio-system -o jsonpath={.items[0].spec.ports[2].nodePort}
    INGRESS_PORT_SSL = int(INGRESS_PORT_SSL_LIST[0])
    GRAFANA_HOST = INGRESS_HOST
    GRAFANA_PORT_LIST = !kubectl -n knative-monitoring get service grafana-load-balancer -o jsonpath={.spec.ports[0].nodePort}
    GRAFANA_PORT = GRAFANA_PORT_LIST[0]

print(f"http://{GRAFANA_HOST}:{GRAFANA_PORT}/d/drTDt1LGz/model-performance?orgId=1&refresh=10s&from=now-5m&to=now")

http://a82edfb6ac8824de7bd54c73290e84ae-2060943740.eu-central-1.elb.amazonaws.com:3000/d/drTDt1LGz/model-performance?orgId=1&refresh=10s&from=now-5m&to=now


# Enable HTTPS

### Generate client and server certificates and keys
Follow: https://istio.io/latest/docs/tasks/traffic-management/ingress/secure-ingress/#generate-client-and-server-certificates-and-keys

In [26]:
 !kubectl create -n istio-system secret tls credential --key=./istio/ssl/httpbin.example.com.key --cert=./istio/ssl/httpbin.example.com.crt

secret/credential created


In [27]:
!kubectl -n knative-serving edit gateway knative-ingress-gateway
# Append:
"""
  - hosts:
    - '*'
    port:
      name: https
      number: 443
      protocol: HTTPS
    tls:
      credentialName: credential
      mode: SIMPLE
"""

gateway.networking.istio.io/knative-ingress-gateway edited


"\n  - hosts:\n    - '*'\n    port:\n      name: https\n      number: 443\n      protocol: HTTPS\n    tls:\n      credentialName: credential\n      mode: SIMPLE\n"

# Enable Token based Authentication
### Generate a public/private key pair

In [28]:
!openssl genrsa -out ./istio/Request_authentication/private-key.pem 2048
!openssl rsa -in ./istio/Request_authentication/private-key.pem -pubout -out ./istio/Request_authentication/public-key.pem

Generating RSA private key, 2048 bit long modulus (2 primes)
........+++++
...+++++
e is 65537 (0x010001)
writing RSA key


### Generate JWT Using the Private Key Generated in Previous Step

In [29]:
"""
This script uses RSA public/private key pair generated using Openssl command line tool.
The series of steps are listed below
1. Import openssl generated public/private key pair
3. Generate the Token using the Private key from step 1
4. Validate the JWT Token using the Public key from step 1
"""
# ______________________________ Step 0 ______________________________________
import python_jwt as jwt, jwcrypto.jwk as jwk, datetime

# ______________________________ Step 1 ______________________________________
# ______________________________ IMPORT KEY ______________________________________
# Import the key.
# The private key will be used to Generate the Token

# Path to the private and public key files generated using openssl
PRIVATE_KEY_FILE="./istio/Request_authentication/private-key.pem"
PUBLIC_KEY_FILE="./istio/Request_authentication/public-key.pem"

# Define payload
# payload that the server will send back the client encoded in the JWT Token
# While generating a token, you can define any type of payload in valid JSON format
# the iss(issuer), sub(subject) and aud(audience) are reserved claims. https://tools.ietf.org/html/rfc7519#section-4.1
# These reserved claims are not mandatory to define in a standard JWT token.
# But when working with Istio, it's better you define these.

payload = {
    'iss':'ISSUER', 
    'sub':'SUBJECT', 
    'aud':'AUDIENCE', 
    'role': 'user', 
    'permission': 'read' 
}


public_key = ""
private_key = ""
token=""

with open(PUBLIC_KEY_FILE, "rb") as pemfile:
    public_key = jwk.JWK.from_pem(pemfile.read())
    public_key = public_key.export()
    


with open(PRIVATE_KEY_FILE, "rb") as pemfile:
    private_key = jwk.JWK.from_pem(pemfile.read())
    private_key = private_key.export()


# ______________________________ Step 2 ______________________________________
# ______________________________ GENERATE JWT TOKEN ______________________________________
# Generate the JWT Tokes using the Private Key
# Provide the payload and the Private Key. RS256 is the Hash used and last value is the expiration time.
# You can set the expiration time according to your need.
# To generate JWT Token, you need the private key as a JWK object
token = jwt.generate_jwt(payload, jwk.JWK.from_json(private_key), 'RS256', datetime.timedelta(days=500000))


# Print the public key, private key and the token
print("\n_________________PUBLIC___________________\n")
print(public_key)
print("\n_________________PRIVATE___________________\n")
print(private_key)
print("\n_________________TOKEN___________________\n")
print(token)



# ______________________________ Step 4 ______________________________________
# ______________________________ VALIDATE JWT TOKEN USING PUBLIC KEY ______________________________________


# To validate JWT Token, you need the public key as a JWK object
header, claims = jwt.verify_jwt(token, jwk.JWK.from_json(public_key), ['RS256'])

print("\n_________________TOKEN INFO___________________\n")
print(header)
print(claims)


_________________PUBLIC___________________

{"e":"AQAB","kid":"y-v2NsDA1v_8qAskIMBsphZ9bU7fg-pcZUF1-mfQRXA","kty":"RSA","n":"xpr6MWGB1-y9PtN7tvZVNyLQbIqDNdm3HmrwuTezdRK-HTWXByfs_cqhYwcRg8SP16apnqm5SoJ9fPXORttoW2IWD8ltnNmlcborYK-JzFBZl9lK6vIJTEEj2zsjFre8oobfIR7TuaJ19BEYBnElfLEwvBAzXE44KZ3hkFn-Y17bqkemy_Vt238tnmXhBoSUiqlIPR-JsAf3zzo-chyW036rMwFlTcKQ-oDpzXRjzAT07m_kDSHgdiq8iM3dyGXI-WQfIb7sbXL5nknE83OZIZDALIJOXIK76PPJtNyVhsS_kHh8fiSfBCoSNHw5aS5atm0QFMSHxLeJX5_9g8cbKQ"}

_________________PRIVATE___________________

{"d":"dYLTUI6KHjGUU-0cAUjFaQcvXVmjgyRbxiKuZlj_1OCPHodL4k8MWaogTZCsG1QdbBLPv_JakSyehWaHc8N0fsWNmi-rrKfWzXDDI8UZeot8R22pd1RYjgbo6VmXTGTQtzWoJlewHOF0e0H1_jHKZXoOBOhtC0u4zV7-TMQC0dYN_bSBGZOT0HFOlWKPZ3qZCJ-VwYFY7AKSHCCKUTWpResQhkAAQGqEUkRBmR6TmXl6UtjLe7VZdbPXPZ3uu5imJOxjyuiWLDjBRpuk4wdmyI65yIgVEIzRdij3OpRlHPLHCcVKCfDujXcnzWiLmFFzPCXA1n3h-CDTUWLownu1NQ","dp":"lHvFVe7GzBCigP8b-pv6Z9RZB4U2f_9hEuoNYmgB79rxffcl56ppI5rV7Eyyh8Dnq1JDDNjdCKXgnMMZN0ypdRJ1lIs0KtYcbY47mlwkUKSWwWymOp_GzRRYJUfJ4J9

### Generate RSA Public/Private Key Pairs and JWT Using Python

In [30]:
"""
This script generates RSA public/private key pair using python.
And uses the Keys to Generate JWT Token.
The series of steps are listed below
1. Generate the Key
2. Generate the Public and Private Keys
3. Generate the Token using the Private Key from step 2
4. Validate the JWT Token using the Public key from step 2
"""


# ______________________________ Step 0 ______________________________________
# import python_jwt
import python_jwt as jwt, jwcrypto.jwk as jwk, datetime

# ______________________________ Step 1 ______________________________________
# ______________________________ GENERATE KEY ______________________________________
# Generate the keys.
# The private key will be used to Generate the Token
# The 2048-bit is about the RSA key pair: 
# RSA keys are mathematical objects which include a big integer, and a "2048-bit key" is a key such that 
# the big integer is larger than 2^2047 but smaller than 2^2048.
key = jwk.JWK.generate(kty='RSA', size=2048)

# Define payload
# payload that the server will send back the client encoded in the JWT Token
# While generating a token, you can define any type of payload in valid JSON format
# the iss(issuer), sub(subject) and aud(audience) are reserved claims. https://tools.ietf.org/html/rfc7519#section-4.1
# These reserved claims are not mandatory to define in a standard JWT token.
# But when working with Istio, it's better you define these.

payload = {
    'iss':'ISSUER', 
    'sub':'SUBJECT', 
    'aud':'AUDIENCE', 
    'role': 'user', 
    'permission': 'read' 
}

# ______________________________ Step 2 ______________________________________
# ______________________________ GENERATE PUBLIC AND PRIVATE KEY ______________________________________
# Export the private and public key

private_key = key.export_private()
public_key = key.export_public()


# ______________________________ Step 3 ______________________________________
# ______________________________ GENERATE JWT TOKEN ______________________________________
# Generate the JWT Tokes using the Private Key
# Provide the payload and the Private Key. RS256 is the Hash used and last value is the expiration time.
# You can set the expiration time according to your need.
# To generate JWT Token, you need the private key as a JWK object
token = jwt.generate_jwt(payload, jwk.JWK.from_json(private_key), 'RS256', datetime.timedelta(days=5000))



# Print the public key, private key and the token
print("\n_________________PUBLIC___________________\n")
print(public_key)
print("\n_________________PRIVATE___________________\n")
print(private_key)
print("\n_________________TOKEN___________________\n")
print(token)




# ______________________________ Step 4 ______________________________________
# ______________________________ VALIDATE JWT TOKEN USING PUBLIC KEY ______________________________________


# To validate JWT Token, you need the public key as a JWK object
header, claims = jwt.verify_jwt(token, jwk.JWK.from_json(public_key), ['RS256'])

print("\n_________________TOKEN INFO___________________\n")
print(header)
print(claims)


_________________PUBLIC___________________

{"e":"AQAB","kty":"RSA","n":"nqP_xN2thmWuz8E9fW7IxhrLp7qZefvjgHeyfNHOfFkzpPS1jkT-8l3WZg138Ej1fglKBu_GHQcY_wydyJjKXYy5yfyW61vAfxXBSiWmtbicS8RE-oroSKmcv6RIrw6ZtwCyCr4rcmu-Bn-a-_0eMedlwYkbROinPxlF8gQX6VAUe_WPJMcY9x1yvWLgHxFjgVWuqzaIj1U_tMg8AxdxIxaFA-j2GH6-RhMUb2k0oJ5UxW9KcJA42gIbSuice7GN6YCKSvUVm2IeSfgLRpAM4liAt8dRUAUUKLw-xsnBxtQLRKeXDAPTxpKsB2pBIvz_sfcnPeqXDUSI-FRKHkXwxQ"}

_________________PRIVATE___________________

{"d":"kvOP5akDKM_gUwrKDvskeH4x0LmFmf2_DC3U5NLk10M6F7-mvpcjIxfRls87HxY2cf3g2PQbLKB6gygIsNz3-Bh3saeNlY90jUR1vF2MRCEyhuzUiNFLwqN7U_q2aZK4yVCXnGP0kxVC_XoO8wXRhqC3HcQHwplQ855RsJIiJDhYoxzWBxs0-xG7LezzXzcTewFuQrXUmASqrqZk97kZSTOBklDptWQTtI8NNKmL7I7a7UTi1iXYHbNGyK3yQndoH_fIznxFsifh8_Q-RNmrzIhkfOIocn1qmbEuxRudS9dtMSPexAXX3Ut2_V2eXgccDGJUnhOhWzhHE6jlh_bAAQ","dp":"DyHTKq-nCSaufpAtDX86AB_2imQeNTEsC0L50EMxMFRoswyL_O1ZnT-Ae_Kv920fIH_KoiI81CzsVjb4QypTgSYk_jOhNrmrL3HaeDTdov65GQabTPPbpefcBU2Ag1F7bg82ehgJ-CGgkuqv8V9ChRckmqXYNMGADSCrC1bteAE","dq":"

### Create a JWKS - JSON Web Key Set

In [31]:
import ast

public_key_dict = ast.literal_eval(public_key)
public_key_dict_array = {'keys': [public_key_dict]}
public_key_dict_array

{'keys': [{'e': 'AQAB',
   'kty': 'RSA',
   'n': 'nqP_xN2thmWuz8E9fW7IxhrLp7qZefvjgHeyfNHOfFkzpPS1jkT-8l3WZg138Ej1fglKBu_GHQcY_wydyJjKXYy5yfyW61vAfxXBSiWmtbicS8RE-oroSKmcv6RIrw6ZtwCyCr4rcmu-Bn-a-_0eMedlwYkbROinPxlF8gQX6VAUe_WPJMcY9x1yvWLgHxFjgVWuqzaIj1U_tMg8AxdxIxaFA-j2GH6-RhMUb2k0oJ5UxW9KcJA42gIbSuice7GN6YCKSvUVm2IeSfgLRpAM4liAt8dRUAUUKLw-xsnBxtQLRKeXDAPTxpKsB2pBIvz_sfcnPeqXDUSI-FRKHkXwxQ'}]}

In [32]:
%%writetemplate ./istio/Request_authentication/RequestAuthentication.yaml
apiVersion: "security.istio.io/v1beta1"
kind: "RequestAuthentication"
metadata:
  name: "jwt-example"
  namespace: istio-system
spec:
  selector:
    matchLabels:
      istio: ingressgateway
  jwtRules:
  - issuer: "ISSUER"
    jwks: |
        {public_key_dict_array}

In [33]:
!kubectl apply -f ./istio/Request_authentication/RequestAuthentication.yaml

requestauthentication.security.istio.io/jwt-example created


### Reject invalid Requests without Token

In [34]:
%%writefile ./istio/Request_authentication/AuthorizationPolicy.yaml
apiVersion: "security.istio.io/v1beta1"
kind: "AuthorizationPolicy"
metadata:
  name: "deny-ingress"
  namespace: istio-system
spec:
  selector:
    matchLabels:
      istio: ingressgateway
  action: DENY
  rules:
  - from:
    - source:
        notRequestPrincipals: ["*"]

Overwriting ./istio/Request_authentication/AuthorizationPolicy.yaml


In [35]:
!kubectl apply -f ./istio/Request_authentication/AuthorizationPolicy.yaml

authorizationpolicy.security.istio.io/deny-ingress created


# Test the ML-Service
### Load Test Data

In [36]:
import pandas as pd
import numpy as np
import time
import json
import requests
import urllib3
from IPython.core.interactiveshell import InteractiveShell

urllib3.disable_warnings()
InteractiveShell.ast_node_interactivity = "all"
np.set_printoptions(precision=5)

sessions_padded = np.load('list_sessions_padded.npy')
print(sessions_padded.shape)

last_clicked = np.load('list_last_clicked.npy')
print(last_clicked.shape)

id_mapping = pd.read_csv('ID_Mapping.csv')

(30941, 30, 52)
(30941,)


In [37]:
def request_kf_serving_http(np_array, ground_truth, MODEL_NAME, NAMESPACE, INGRESS_HOST, INGRESS_PORT, token):
    data = json.dumps({"instances": np_array.tolist(),
                       'id': ground_truth.tolist()})
    
    headers = {"content-type": "application/json",
               'Host': f'{MODEL_NAME}.{NAMESPACE}.example.com',
               "Authorization": f"Bearer {token}"}
    
    json_response = requests.post(
        f'http://{INGRESS_HOST}:{INGRESS_PORT}/v1/models/{MODEL_NAME}:predict',
        data=data, headers=headers)

    try:
        predictions = json.loads(json_response.text)['predictions']
    except Exception as e:
        print(json_response.text)
        raise e
    return np.array(predictions).astype(np.float32)


def request_kf_serving_https(np_array, ground_truth, MODEL_NAME, NAMESPACE, INGRESS_HOST, INGRESS_PORT_SSL, token):
    data = json.dumps({"instances": np_array.tolist(),
                       'id': ground_truth.tolist()})
    headers = {"content-type": "application/json",
               'Host': f'{MODEL_NAME}.{NAMESPACE}.example.com',
               "Authorization": f"Bearer {token}"}
    json_response = requests.post(
        f'https://{INGRESS_HOST}:{INGRESS_PORT_SSL}/v1/models/{MODEL_NAME}:predict',
        data=data, headers=headers, verify=False)

    try:
        predictions = json.loads(json_response.text)['predictions']
    except Exception as e:
        print(json_response.text)
        raise e
    return np.array(predictions).astype(np.float32)


NAMESPACE = 'kfserving-test'
MODEL_NAME = 'product-recommender'

## HTTP Request

In [39]:
idx = 14 # 15, 169, 14 anomaly: 169

start = time.time()
pred = request_kf_serving_http(sessions_padded[idx:idx+1], last_clicked[idx:idx+1],
                          MODEL_NAME, NAMESPACE, INGRESS_HOST, INGRESS_PORT, token)
end = time.time()
print(f'Time required in Seconds: {end - start}')

top = pred.argsort()[0][::-1][:5]
print("Session:")
session = pd.DataFrame()
session['category_code'] = [id_mapping['category_code'][int(i)-1] for i in sessions_padded[idx,:,0] if i>0]
session['Item_ID'] = [id_mapping['Item_ID'][int(i)-1] for i in sessions_padded[idx,:,0] if i>0]
session['Item_ID_Mapped'] = [int(i) for i in sessions_padded[idx,:,0] if i>0]
session


print("Prediction:")
prediction = pd.DataFrame()
prediction['category_code'] = [id_mapping['category_code'][int(i)-1] for i in top if i>0]
prediction['Item_ID'] = [id_mapping['Item_ID'][int(i)-1] for i in top if i>0]
prediction['Item_ID_Mapped'] = [int(i) for i in top if i>0]
prediction['probability'] = pred[0, top]
prediction
print("Ground Truth:", last_clicked[idx])

Time required in Seconds: 0.2382814884185791
Session:


,category_code,Item_ID,Item_ID_Mapped
0,construction.tools.light,1004617.0,43
1,electronics.smartphone,1005196.0,44
2,construction.tools.light,1004617.0,43
3,construction.tools.light,1005239.0,45
4,appliances.kitchen.refrigerators,1005174.0,46


Prediction:


,category_code,Item_ID,Item_ID_Mapped,probability
0,construction.tools.light,1004785.0,134,0.083772
1,construction.tools.light,1004849.0,668,0.059975
2,construction.tools.light,1002540.0,355,0.053024
3,construction.tools.light,1005169.0,344,0.033094
4,construction.tools.light,1004850.0,836,0.032421


Ground Truth: 43


## HTTPS Request

In [40]:
idx = 15 # 15, 14 anomaly: 169

start = time.time()
pred = request_kf_serving_https(sessions_padded[idx:idx+1], last_clicked[idx:idx+1],
                          MODEL_NAME, NAMESPACE, INGRESS_HOST, INGRESS_PORT_SSL, token)
end = time.time()
print(f'Time required in Seconds: {end - start}')

top = pred.argsort()[0][::-1][:5]
print("Session:")
session = pd.DataFrame()
session['category_code'] = [id_mapping['category_code'][int(i)-1] for i in sessions_padded[idx,:,0] if i>0]
session['Item_ID'] = [id_mapping['Item_ID'][int(i)-1] for i in sessions_padded[idx,:,0] if i>0]
session['Item_ID_Mapped'] = [int(i) for i in sessions_padded[idx,:,0] if i>0]
session


print("Prediction:")
prediction = pd.DataFrame()
prediction['category_code'] = [id_mapping['category_code'][int(i)-1] for i in top if i>0]
prediction['Item_ID'] = [id_mapping['Item_ID'][int(i)-1] for i in top if i>0]
prediction['Item_ID_Mapped'] = [int(i) for i in top if i>0]
prediction['probability'] = pred[0, top]
prediction
print("Ground Truth:", last_clicked[idx])

Time required in Seconds: 0.28354787826538086
Session:


,category_code,Item_ID,Item_ID_Mapped
0,appliances.environment.vacuum,3700338.0,47
1,appliances.environment.vacuum,3700338.0,47
2,appliances.environment.vacuum,3701056.0,48
3,appliances.environment.vacuum,3700338.0,47
4,appliances.environment.vacuum,3701056.0,48
5,appliances.environment.vacuum,3700338.0,47
6,appliances.environment.vacuum,3701056.0,48
7,appliances.environment.vacuum,3700278.0,49
8,appliances.environment.vacuum,3700777.0,50
9,appliances.environment.vacuum,3701162.0,51


Prediction:


,category_code,Item_ID,Item_ID_Mapped,probability
0,appliances.environment.vacuum,3700600.0,3444,0.641037
1,appliances.environment.vacuum,3700832.0,2898,0.045066
2,appliances.environment.vacuum,3700787.0,1691,0.044069
3,appliances.environment.vacuum,3700907.0,406,0.037994
4,appliances.environment.vacuum,3701164.0,293,0.017183


Ground Truth: 3444


# Test Concept Drift Detection

### Anomalies from the Dataset

In [41]:
index_anomalies = [169,   246,   394,   498,   630,  1039,  1578,  2008,  2040,
                   2447,  2557,  2609,  3179,  3276,  3481,  3615,  3813,  4179,
                   4361,  4794,  5077,  6184,  6369,  7347,  7596,  8415,  8761,
                   8773,  9011,  9404,  9504,  9613,  9880,  9907,  9978, 10050,
                   10229, 10573, 10654, 11196, 11429, 11477, 11493, 11654, 11975,
                   12135, 13526, 13659, 13729, 14139, 14469, 14910, 15203, 15429,
                   15934, 15982, 16310, 16352, 16504, 16647, 16743, 17046, 17085,
                   17302, 17342, 17449, 18584, 18702, 18711, 18770, 19204, 19642,
                   19758, 19863, 19891, 20135, 20244, 20652, 20865, 20899, 21077,
                   21680, 23338, 23407, 23892, 24101, 24257, 24259, 24396, 25078,
                   25127, 25380, 25576, 26071, 26082, 26123, 26323, 26373, 27007,
                   27629, 27664, 27833, 28388, 28739, 29576, 29588, 30381, 30529,
                   30873, 30930]

start = time.time()

for i in range(10):
    pred = request_kf_serving_http(sessions_padded[index_anomalies[i*10:(i+1)*10]],
                              last_clicked[index_anomalies[i*10:(i+1)*10]],
                              MODEL_NAME, NAMESPACE, INGRESS_HOST, INGRESS_PORT, token)
end = time.time()
print(f'Time required in Seconds: {end - start}')

Time required in Seconds: 6.582572937011719


## Grafana Dashboard

In [42]:
print(f"http://{GRAFANA_HOST}:{GRAFANA_PORT}/d/drTDt1LGz/model-performance?orgId=1&refresh=10s&from=now-5m&to=now")

http://a82edfb6ac8824de7bd54c73290e84ae-2060943740.eu-central-1.elb.amazonaws.com:3000/d/drTDt1LGz/model-performance?orgId=1&refresh=10s&from=now-5m&to=now
